This project is to book a dentist through python selenium
Author : Wake Liu

In [163]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time

由於每個ID都是獨立的，我們可以在頁面中找到自己要的ID資訊。
以輕鬆美膚網為例，在登入時所需要的資料為ID, Password, Login, 從網站原始碼截取的資訊如下

In [164]:
def login_proc(username, password):
    sel = webdriver.Chrome()
    sel.get("https://www.ezskin.com.tw/member/member_login.aspx")
    # sign in the username
    try:
        user_input = sel.find_element_by_id("ctl00_ContentPlaceHolder1_txtUserID")
        user_input.clear()
        user_input.send_keys(username)
        print ('user-id write success!')
    except:
        print ('user-id write error!')
    # sign in the pasword
    try:
        pwd_input = sel.find_element_by_id("ctl00_ContentPlaceHolder1_txtPassword")
        pwd_input.clear()
        pwd_input.send_keys(password)
        print ('pw write success!')
    except:
        print ('pw write error!')
    # Check for Login success
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnLogin").click()
    except:
        print ('Login Failed')

    while True:
        curpage_url = sel.current_url
        if curpage_url != login_url:
            if curpage_url[:-1] != login_url:  # choose wrong verify_pic
                print ('Login finished!')
                break
        else:
            time.sleep(5)
            print ('Login Failed!')
    return sel

In [165]:
sel = login_proc("F227268068", "lovewake2018")

user-id write success!
pw write success!
Login finished!


Since 門診掛號沒有id,這邊用Xpath的方式將該element 找到
Find the Xpath of 門診掛號

In [166]:
try:
    sel.find_element_by_xpath('//*[@id="body"]/div[4]/div[2]/div[1]/div/div[2]/table/tbody/tr/td/table/tbody/tr[4]/td/div/input').click()
except BaseException:
    print ("無法門診掛號")

仔細觀察，該checkbox有固定的格式
由於不確定之後是109還是110所以兩個都進行檢查
如果有出現該id就直接click然後break 離開迴圈

In [167]:
def select_proc():
    flag = 0
    sel.refresh
    for i in range (1,4):
        checkbox = 'ctl00_ContentPlaceHolder1_dbOutpatient_ctl10_chkSelect'+str(i)
        try:   
            sel.find_element_by_id(checkbox).click()
            flag = 1
            break
        except BaseException:
            continue
            #print(checkbox + " element not exist")
    for i in range (1,4):
        checkbox = 'ctl00_ContentPlaceHolder1_dbOutpatient_ctl09_chkSelect'+str(i)
        try:   
            sel.find_element_by_id(checkbox).click()
            flag = 1
            break
        except BaseException:
            continue
            #print(checkbox + " element not exist")
    return flag

這個步驟是等待，我們只有在23時59分才需要開啟該程式，其他時間都可以休息
如果不確定localtime的語法可以輸入
print ("本地时间为 :", localtime)

In [168]:
while(1):
    localtime = time.localtime(time.time())
    if localtime.tm_hour == 23 and localtime.tm_min == 59:
        break
    print (localtime.tm_min,localtime.tm_sec)
    sleep(50)

56 16
57 6
57 56
58 46


###等待

執行為期兩分鐘的等待時間
每隔0.1秒刷新網頁

In [169]:
for i in range(1200):
    flag = select_proc()
    if flag == True:
        print ("Successfully Selected")
        break
    elif i % 100 ==0:
        print ("Keep waiting")

        sleep (0.1)

Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting
Keep waiting


下一步跟確認掛號都算是簡單的
如果有成功掛號(即flag為true)
只要抓到id繼續click就可以了

<input type="submit" name="ctl00$ContentPlaceHolder1$btnNext" value="下一步" onclick="return checkModify();" id="ctl00_ContentPlaceHolder1_btnNext" class="btn01">

確認掛號語法

<input type="submit" name="ctl00$ContentPlaceHolder1$btnSave" value="確定掛號" id="ctl00_ContentPlaceHolder1_btnSave" class="btn01">

In [170]:
if flag == True:
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnNext").click()
        print ("Successfully Next Step")
    except BaseException:
        print ("cannot press next step")
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnSave").click()
        print ("Successfully Save")
    except BaseException:
        print ("Cannot save")
    if '應用程式中發生伺服器錯誤' not in sel.page_source:
        print ("Your reservation is done")
    else:
        print ("The website error")
else:
    print ("Sorry all the doctors are reserved")

Sorry all the doctors are reserved
